In [1]:
from model.common import CPLX
import torch
from scipy.constants import pi
import matplotlib.pyplot as plt
import numpy as np

In [2]:
device = torch.device('cuda:0')
print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 3090


In [60]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from scipy.constants import pi, c

class FMCW():
    def __init__(self, Nk=64, B=4e9, fs=2000e3, f0=77e9):
        self.Nk = Nk
        self.B = B
        self.fs = fs
        self.f0 = f0
        self.K = fs*B/(Nk-1)
        self.c = c
        
        self.f = f0 + np.arange(self.Nk)*self.K/self.fs
        self.k = 2*pi*self.f/self.c
        
        self.range_max_m = self.fs*self.c/(2*self.K)
        self.range_res_m = self.c/(2*self.B)
        self.lambda_m = self.c/(self.f0 + self.B/2)        

class Radar():
    def __init__(self, Nk=64, B=4e9, fs=2000e3, f0 = 77e9):
        self.fmcw = FMCW(Nk, B, fs, f0)
        
class TIRadar():
    def __init__(self, f0=77e9, K=124.996e12, Nk=64, fS=2000e3):
        self.f0 = f0
        self.K = K
        self.Nk = Nk
        self.fS = fS
        self.Compute()
        
    def __str__(self):
        return f"TI Radar with f0={self.f0*1e-9} GHz, K={self.K*1e-12} MHz/us, Nk={self.Nk}, fS={self.fS*1e-3} ksps"
        
    def Compute(self):
        self.f = self.f0 + self.K/self.fS*(torch.arange(self.Nk))
        self.k = 2*pi/c*self.f
        self.range_max_m = self.fS*c/(2*self.K)
        
class DualRadar():
    def __init__(self, f0=[60e9, 76.999456e9], K=124.996e12, Nk=64, Nk_fb=336, fS=2000e3):
        self.f0 = f0
        self.K = K
        self.Nk = Nk
        self.Nk_fb = Nk_fb
        self.fS = fS
        self.Compute()
        
    def __str__(self):
        return f"Dual Radar with \nradar1: \t{self.r0.__str__()}, \nradar2: \t{self.r1.__str__()} \nfull-band: \t{self.r_fb.__str__()}"
    
    def Compute(self):
        self.r0 = TIRadar(f0=self.f0[0])
        self.r1 = TIRadar(f0=self.f0[1])
        self.r_fb = TIRadar(f0=self.f0[0],Nk=self.Nk_fb)
        
        self.f_LR = torch.zeros(self.Nk_fb)
        self.f_LR[:self.Nk] = self.r0.f
        self.f_LR[-self.Nk:] = self.r1.f
        self.k_LR = 2*pi/c*self.f_LR
        
        self.f_HR = self.r_fb.f
        self.k_HR = self.r_fb.k
        
        
dr = DualRadar()
print(dr)

print(dr.f_HR - dr.f_LR)

Dual Radar with 
radar1: 	TI Radar with f0=60.00000000000001 GHz, K=124.996 MHz/us, Nk=64, fS=2000.0 ksps, 
radar2: 	TI Radar with f0=76.99945600000001 GHz, K=124.996 MHz/us, Nk=64, fS=2000.0 ksps 
full-band: 	TI Radar with f0=60.00000000000001 GHz, K=124.996 MHz/us, Nk=336, fS=2000.0 ksps
tensor([ 0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
         0.0000e+00,  0.0000e+00,  0.0000e+00,  0.00

In [61]:
print(dr.r1.range_max_m)

2.3984164133252266


In [188]:
import torch
from scipy.constants import pi
import numpy as np
import matplotlib.pyplot as plt
from model.common import CPLX

class Data:
    def __init__(self, dr):
        self.X = None
        self.dr = dr
        
    def create_dataset_train(self, num_train, max_targets, xyz_str):
        self.max_targets = max_targets
        
        self.num_train = num_train
        
        self.create_XY(num_train)
        
        self.train_X = self.X
        self.train_Y = self.Y
        self.train_Z = self.Z
        
        X = self.X.reshape((num_train, 1, self.dr.Nk_fb))
        if xyz_str.lower() == 'xy':
            Y = self.Y.reshape((num_train, 1, self.dr.Nk_fb))
        elif xyz_str.lower() == 'xz':
            Y = self.Z.reshape((num_train, 1, self.dr.Nk_fb))
        
        X = torch.cat((torch.real(X), torch.imag(X)), axis=1)
        Y = torch.cat((torch.real(Y), torch.imag(Y)), axis=1)
        
        self.dataset_train = torch.utils.data.TensorDataset(X, Y)
        
    def create_dataset_test(self, num_test, max_targets, xyz_str):
        self.max_freqs = max_targets
        
        self.num_test = num_test
        
        self.create_XY(num_test)
        
        self.test_X = self.X
        self.test_Y = self.Y
        self.test_Z = self.Z
        
        X = self.X.reshape((num_test, 1, self.dr.Nk_fb))
        if xyz_str =='xy':
            Y = self.Y.reshape((num_test, 1, self.dr.Nk_fb))
        elif xyz_str == 'xz':
            Y = self.Z.reshape((num_test, 1, self.dr.Nk_fb))
        
        X = torch.cat((torch.real(X), torch.imag(X)), axis=1)
        Y = torch.cat((torch.real(Y), torch.imag(Y)), axis=1)
        
        self.dataset_test = torch.utils.data.TensorDataset(X, Y)
       
    def create_XY(self, num):
        def linear_power(x, N):
            return torch.sqrt(1/N * torch.sum(torch.abs(x)**2))
        
        X = torch.zeros((num, self.dr.Nk_fb), dtype=torch.complex64)
        Y = torch.zeros((num, self.dr.Nk_fb), dtype=torch.complex64)
        Z = torch.zeros((num, self.dr.Nk_fb), dtype=torch.complex64)
        SNR = np.zeros(num)
        
        k = self.dr.k_HR.reshape((1, self.dr.Nk_fb))
        
        for ind_num in range(num):
            # Handle noise
            SNR[ind_num] = 50*np.random.rand(1)
            n = torch.randn((2*self.dr.Nk), dtype=torch.complex64)
            n_pow = linear_power(n, 2*self.dr.Nk)
            
            # Handle signal
            num_targets = np.random.randint(1, self.max_targets+1)
            R = 0.9*self.dr.r0.range_max_m*torch.rand(num_targets, 1).reshape((num_targets, 1))
            
            amps = torch.randn(num_targets, 1).reshape((num_targets, 1))
            theta = 2*pi*torch.rand(num_targets, 1).reshape((num_targets, 1))
            
            amps = (0.1 + torch.abs(amps)) * torch.exp(1j*theta)
            
            x_HR = torch.sum(amps * torch.exp(1j*2*k*R), axis=0)
            x_LR = x_HR.clone()
            x_LR[self.dr.Nk:-self.dr.Nk] = torch.zeros(self.dr.Nk_fb - 2*self.dr.Nk, dtype=torch.complex64)
            
            x_LR_pow = linear_power(x_LR, 2*self.dr.Nk)
            x_HR_pow = linear_power(x_HR, self.dr.Nk_fb)
            
            n = n/n_pow*x_LR_pow*10**(-SNR[ind_num]/10)
            
            x_LR[:self.dr.Nk] = n[:self.dr.Nk]
            x_LR[-self.dr.Nk:] = n[-self.dr.Nk:]
            
            x_LR_pow = linear_power(x_LR, 2*self.dr.Nk)
            x_HR_pow = linear_power(x_HR, self.dr.Nk_fb)
            
            x_LR = x_LR/x_LR_pow
            x_HR = x_HR/x_HR_pow
            
            y_LR = torch.fft.fft(x_LR, self.dr.Nk_fb, dim=0, norm="ortho")
            y_HR = torch.fft.fft(x_HR, self.dr.Nk_fb, dim=0, norm="ortho")
            
            X[ind_num] = y_LR
            Y[ind_num] = y_HR
            Z[ind_num] = x_HR
            
        self.X = X
        self.Y = Y
        self.Z = Z
        
    def test_net(self, args, model, data):
        ind_rand = np.random.randint(0, data.num_test)

        (X, Y) = data.dataset_test[ind_rand]
        x_in = X.to(args.device)

        x_in = CPLX(x_in[0, :].reshape((1, 1, -1)), x_in[1, :].reshape((1, 1, -1)))
        Y_pred = model.model(x_in)
        y_pred = (Y_pred.r + 1j*Y_pred.i).cpu().detach().numpy().reshape(-1)

        x = X[0] + 1j*X[1]
        y = Y[0] + 1j*Y[1]
        
        if args.xyz_str.lower() == 'xz':
            y = np.fft.fft(y, norm="ortho")
            y_pred = np.fft.fft(y_pred, norm="ortho")
        else:
            y = y.numpy()

        fig = plt.figure(figsize=[24, 16])
        plt.subplot(241)
        plt.plot(np.abs(x))
        plt.title("Input " + str(ind_rand))

        plt.subplot(242)
        plt.plot(np.abs(y_pred))
        plt.title("Output")

        plt.subplot(243)
        plt.plot(np.abs(y))
        plt.title("Ground Truth")

        plt.subplot(244)
        err = np.abs(y - y_pred)
        plt.plot(err)
        plt.title("Error")

        x = np.fft.ifft(x, norm="ortho")
        y = np.fft.ifft(y, norm="ortho")
        y_pred = np.fft.ifft(y_pred, norm="ortho")            
            
        plt.subplot(245)
        plt.plot(np.real(x))
        plt.title("Input")

        plt.subplot(246)
        plt.plot(np.real(y_pred))
        plt.title("Output")

        plt.subplot(247)
        plt.plot(np.real(y))
        plt.title("Ground Truth")

        plt.subplot(248)
        err = y - y_pred
        plt.plot(np.real(err))
        plt.title("Error")

        plt.show()
        
        
dr = DualRadar()
d = Data(dr)
d.create_dataset_train(256, 64, "xz")


In [1]:


fig = plt.figure()
plt.plot(torch.abs(d.train_X[1, :]))
plt.show()

fig = plt.figure()
plt.plot(torch.abs(d.train_Y[1, :]))
plt.show()

fig = plt.figure()
plt.plot(torch.real(d.train_Z[1, :]))
plt.show()

d.create_dataset_test(256, 64, "xz")

fig = plt.figure()
plt.plot(torch.abs(d.test_X[1, :]))
plt.show()

fig = plt.figure()
plt.plot(torch.abs(d.test_Y[1, :]))
plt.show()

fig = plt.figure()
plt.plot(torch.real(d.test_Z[1, :]))
plt.show()

NameError: name 'plt' is not defined

In [52]:
import torch
from scipy.constants import pi
import numpy as np
import matplotlib.pyplot as plt
from model.common import CPLX
import scipy.io as sio

selfargsdevice = torch.device("cuda:0")

def load_matlab_data(
    filename_str=["cutout1", "cutout2", "circle", "diamond", "square", "star", "triangle", "knife"]
):
    d = {}
    num_samples = 0
    for filename in filename_str:
        d[filename] = sio.loadmat(f"./matlab_data/data/{filename}.mat")['sarData']
        N = d[filename].shape[0]
        Nk = d[filename].shape[1]
        num_samples += N
        
    data_all = np.zeros((num_samples, Nk), dtype=np.complex64)
        
    for ind, key in enumerate(d):
        ind_start = N*ind
        ind_end = N*(ind+1)
        data_all[ind_start:ind_end, :] = d[key]
        
    del(d)
        
    return data_all, num_samples

def create_datasets_matlab(
    self, 
    xyz_str, 
    is_noise=True,
    split=[200000, 1000, 1000],
    filename_str=["cutout1", "cutout2", "circle", "diamond", "square", "star", "triangle", "knife"]
    ):
    def linear_power(x, N):
        return torch.sqrt(1/N * torch.sum(torch.abs(x)**2))
    
    
    data_all, num_samples = load_matlab_data(filename_str=filename_str)
    
    x_HR = torch.from_numpy(data_all)
    x_LR = x_HR.clone()
    x_LR[:,self.dr.Nk:-self.dr.Nk] = torch.zeros((num_samples, self.dr.Nk_fb - 2*self.dr.Nk), dtype=torch.complex64)
    
    # Normalize power of all HR samples
    for ind, hr in enumerate(x_HR):
        hr_pow = linear_power(hr, self.dr.Nk_fb)
        x_HR[ind, :] = hr/hr_pow
        
    # Normalize power of all LR samples
    for ind, lr in enumerate(x_LR):
        lr_pow = linear_power(lr, 2*self.dr.Nk)
        x_LR[ind, :] = lr/lr_pow
    
    # Add Guassian noise to LR samples
    if is_noise:
        SNR = np.zeros(num_samples)
        for ind, lr in enumerate(x_LR):
            # Select SNR from 0 dB - 50 dB
            SNR[ind] = 50*np.random.rand(1)
            
            # Generate noise
            n = torch.randn((2*self.dr.Nk), dtype=torch.complex64)
            n_pow = linear_power(n, 2*self.dr.Nk)
            
            # Get power of lr (should be 1)
            lr_pow = linear_power(lr, 2*self.dr.Nk)
            
            # Scale noise power
            n = n/n_pow*lr_pow*10**(-SNR[ind]/10)
            
            # Add noise to signal
            lr[:self.dr.Nk] += n[:self.dr.Nk]
            lr[-self.dr.Nk:] += n[-self.dr.Nk:]
            
            # Set signal back
            x_LR[ind, :] = lr
            
    X = torch.fft.fft(x_LR, self.dr.Nk_fb, dim=-1, norm="ortho")
    Y = x_HR
    
    # Get either xy or xz:
    if xyz_str.lower() =='xy':
        Y = Y.reshape((num_samples, 1, self.dr.Nk_fb))
    elif xyz_str.lower() == 'xz':
        Y = torch.fft.fft(Y, self.dr.Nk_fb, dim=-1, norm="ortho")
        Y = Y.reshape((num_samples, 1, self.dr.Nk_fb))
        
    # Shuffle data
    ind_shuffle = np.random.permutation(num_samples)
    X = X[ind_shuffle, :, :]
    Y = Y[ind_shuffle, :, :]
    SNR = SNR[ind_shuffle]

    # Reshape (num_samples x 2 x Nk_fb)
    X = torch.cat((torch.real(X), torch.imag(X)), axis=1)
    Y = torch.cat((torch.real(Y), torch.imag(Y)), axis=1)
    
    # Split data
    self.train_X = X[:split[0], :, :]
    self.train_Y = Y[:split[0], :, :]
    self.train_SNR = SNR[:split[0]]
    self.dataset_train = torch.utils.data.TensorDataset(self.train_X.to(self.args.device), self.train_Y.to(self.args.device))
    
    self.val_X = X[split[0]:(split[0]+split[1]), :, :]
    self.val_Y = Y[split[0]:(split[0]+split[1]), :, :]
    self.val_SNR = SNR[split[0]:(split[0]+split[1])]
    self.dataset_val = torch.utils.data.TensorDataset(self.val_X.to(self.args.device), self.val_Y.to(self.args.device))
    
    self.test_X = X[(split[0]+split[1]):(split[0]+split[1]+split[2]), :, :]
    self.test_Y = X[(split[0]+split[1]):(split[0]+split[1]+split[2]), :, :]
    self.test_SNR = SNR[(split[0]+split[1]):(split[0]+split[1]+split[2])]
    self.dataset_test = torch.utils.data.TensorDataset(self.test_X, self.test_Y)
    
    self.num_train, self.num_val, self.num_test = split[0], split[1], split[2]
    
    

In [54]:
d = load_matlab_data()

In [58]:
print(d[0].shape)

(524288, 336)


tensor([0.4433+0.0340j])
